In [9]:
import pandas as pd
import os
import glob

In [2]:
from Bio import SeqIO

In [4]:
for file in os.listdir('/home/nastya/Bacteroides_uniformis_annotations/fasta_ncbi'):
    print(file)

GCF_009025335.1_ASM902533v1_cds_from_genomic.fna
GCF_000011065.1_ASM1106v1_cds_from_genomic.fna
GCF_003472675.1_ASM347267v1_cds_from_genomic.fna
GCF_020558235.1_ASM2055823v1_cds_from_genomic.fna
GCF_016760165.1_ASM1676016v1_cds_from_genomic.fna
GCF_007896575.1_ASM789657v1_cds_from_genomic.fna
GCF_021410325.1_ASM2141032v1_cds_from_genomic.fna
GCF_003438545.1_ASM343854v1_cds_from_genomic.fna
GCF_900624775.1_82B11_cds_from_genomic.fna
GCF_023668085.1_ASM2366808v1_cds_from_genomic.fna
GCF_001692695.1_ASM169269v1_cds_from_genomic.fna
GCF_003439865.1_ASM343986v1_cds_from_genomic.fna
GCF_019128965.1_ASM1912896v1_cds_from_genomic.fna
GCF_015558535.1_ASM1555853v1_cds_from_genomic.fna
GCF_000598365.1_ASM59836v1_cds_from_genomic.fna
GCF_002632415.1_ASM263241v1_cds_from_genomic.fna
GCF_009020545.1_ASM902054v1_cds_from_genomic.fna
GCF_903181445.1_NZ_Bacteroides_fragilis_8E3_BL_hyb_cds_from_genomic.fna
GCF_900624705.1_81A2_cds_from_genomic.fna
GCF_009024785.1_ASM902478v1_cds_from_genomic.fna
GCF_023

In [6]:
sum_table = pd.read_csv('data_summary_for_upstream.csv')[['Organism Scientific Name', 'Assembly Accession']]
sum_table

,Organism Scientific Name,Assembly Accession
0,Bacteroides helcogenes P 36-108,GCF_000186225.1
1,Bacteroides fluxus YIT 12057,GCF_000195635.1
2,Bacteroides coprosuis DSM 18011,GCF_000212915.1
3,Bacteroides gallinarum DSM 18171 = JCM 13658,GCF_000374365.1
4,Bacteroides propionicifaciens DSM 19291 = JCM ...,GCF_000375405.1
5,Bacteroides graminisolvens DSM 19988 = JCM 15093,GCF_000428125.1
6,Bacteroides timonensis,GCF_000513195.1
7,Bacteroides rodentium JCM 16496,GCF_000614125.1
8,Bacteroides ovatus,GCF_001314995.1
9,Bacteroides salyersiae,GCF_001405695.1


In [6]:
import json
import os
import subprocess
import time
from Bio import  Entrez, pairwise2, Seq, SearchIO
from Bio.SeqRecord import SeqRecord
from io import StringIO


def split_accession(acc):
    
    """ Splits a genbank/refseq record into prefix (e.g. NZ_MDWE or MDWE) and
        numeric component (e.g. 0100092), and returns them as a list.
    """

    #go through the record until we get the first numeric digit
    #store the prefix and use its length to get the numeric region
    prefix = ''
    for c in acc:
        if c.isdigit():
            break
        else:
            prefix=prefix + c

    suffix = acc[len(prefix):len(acc)]

    return [prefix, suffix]


def genome_record_retrieval(ortholog_acc, sleepy):
    
    """ Takes a protein accession as an input. Retrieves its IPG record.
        The idea here is to obtain, prioritarily, data from complete genome 
        records if they exist, from RefSeq (AC_ and NC_ accessions) . If no 
        RefSeq is available, then select complete genome records from GenBank 
        (AE, CP, CY accessions). Otherwise, select contigs or WGS scaffolds from 
        RefSeq (NT_, NW_, NZ_). If that fails, get contigs or WGS scaffolds from 
        GenBank (AAAA-AZZZ). Only when nothing else is available, select direct 
        GenBank submissions (U, AF, AY, DQ).
        Prioritizes each type of accession and returns the best record.
        Priority indices range from 7 (best, for a complete RefSeq record) and 6
        (complete GenBank record), to 5 (for complete RefSeq WGS) and all the 
        way to 3 (undetermined GenBank records).
        
        It returns a composite record with the nucleotide accession number for
        the "best" coding region, and the position and orientation of the CDS 
        within that accession, as well as the prioritization score obtained.
    """

    #Download IPG record for the specific ortholog
    records = None
    while records == None:
        try:
            records = Entrez.read(Entrez.efetch(db="protein", id=ortholog_acc, \
                                        rettype='ipg', retmode='xml'))
        except:
            print ("IPG record retrieval error: "+ortholog_acc)
            time.sleep(sleepy)
            pass
    
    #create scoring for priorization
    priority = {"NC_": 7, "AC_": 7, "AE": 6, "CP": 6, "CY": 6, \
                    "NZ_": 5, "NT_": 5, "NW_": 5, "AAAA-AZZZ": 4,\
                    "U": 3, "AF": 3, "AY": 3, "DQ": 3}
    
    #from the IPG record, retrieve all the genome accessions from all CDS
    #keeping only accession, location of start and strand, as well as 
    #priority score
    genomelist = []
    if 'ProteinList' in records["IPGReport"].keys():
        for idprotein in records["IPGReport"]["ProteinList"]:
            if 'CDSList' in idprotein.keys():
                for cds in idprotein['CDSList']:
                    cds_acc = cds.attributes['accver']
                    cds_start = cds.attributes['start']
                    cds_stop = cds.attributes['stop']
                    cds_strand = cds.attributes['strand']
                    cds_org = cds.attributes['org'].replace(" ","_")
                    cds_scr = 0
                    
                    #assign priority
                    for key in priority:
                        if cds_acc.startswith(key):
                            cds_scr = priority[key]
                            
                    #special case: NZ_ records that map to "complete" WGS
                    #these are NZ_ records with a short numeral
                    #e.g. NZ_CP030158
                    #the idea is to bump them up to 6, so they
                    #are treated as complete
                    if cds_acc.startswith('NZ_'):
                        pr,sf = split_accession(cds_acc)
                        #non-complete WGS acc seem to have alwayas 8 numericals
                        #so we pick as "complete" anything with less than 7
                        if len(sf.split('.')[0])<7:
                            cds_scr = 6.5
                    
                    #create and append record
                    cds_rec = {'acc':cds_acc, 'start':cds_start, \
                               'stop':cds_stop, 'strand':cds_strand,\
                               'p_score':cds_scr, 'org':cds_org}
                    genomelist.append(cds_rec)
            else:
                return (None)
    #GenBank record that has no proper IPG record (yes, they exist;
    #see for instance: https://www.ncbi.nlm.nih.gov/protein/RJR51       119.1)
    #in these cases, there should be a CDS within the protein record that 
    #contains the information we want; priority should be lowest
    else:
#        TO BE IMPLEMENTED
#        records = Entrez.read(Entrez.efetch(db="protein", id=ortholog_acc, \
#                                            rettype='genbank', retmode='xml'))
        return (None)
        
    #select the genomes with highest value (in case of same scores, 
    #first one is chosen (random)
    max_record = genomelist[0]
    for genome in genomelist:
        if genome['p_score'] >= max_record['p_score']:
            max_record = genome     
    return max_record
    
    time.sleep(sleepy)


def promoter_retrieval(nucid,start,stop,strand,start_adj=250,stop_adj=2, min_size = 50):
    
    """Download promoter sequences (only intergenic) from NCBI RefSeq/GenBank 
       database using the RefSeq/GenBank ID; and the coordinates & strand of 
       gene of interest.
       The start_adj and stop_adj delimit the promoter size.
       Returns only the seq of SeqRecord object
    """
    
    if strand == "+":
        s_start=int(start)-start_adj
        s_stop=int(start)+stop_adj
        s_strand=1
    else:
        s_stop=int(stop)+start_adj
        s_start=int(stop)-stop_adj
        s_strand=2
    
    #Fetch and read the annotated GenBank record
    handle = None
    while handle == None:
        try: 
            handle = Entrez.efetch(db="nuccore",id=nucid, strand=s_strand,seq_start=s_start, seq_stop=s_stop, rettype='gbwithparts', retmode="XML")
            genome_record = Entrez.read(handle, "xml")
        except:
            time.sleep(5)
            print ("Promoter retrieval error: "+nucid)
            pass
    
    # Find all coding regions in the returned GenBank sequence. 
    coding_intervals = []
    sequence = genome_record[0]['GBSeq_sequence']
    for feature in genome_record[0]['GBSeq_feature-table']:
        if feature['GBFeature_key'] == "CDS": 
            if "GBInterval_from" in feature['GBFeature_intervals'][0]:
                coding_start = feature['GBFeature_intervals'][0]['GBInterval_from']
                coding_end = feature['GBFeature_intervals'][0]['GBInterval_to']
                coding_intervals.append((coding_start, coding_end))  
            
    #The FASTA ID for the promoter sequence is in the following format:
        # p_NucleotideRecord
    return_id = "p_" + str(nucid)
    #If there is only one coding region in the selected sequence, then 
    # the sequence is returned unmodified. 
    if len(coding_intervals) == 1:
        #Appends information to record description
        return SeqRecord(Seq.Seq(sequence), id= return_id, description = return_id+"_"+str(s_start)+"_"+str(s_stop))
    #If no coding intervals are indentified, None is returned.
    elif len(coding_intervals) == 0:
        return None
    #The start of the promoter is set to the start/end of the upstream gene
    # based on the directionality. ( --> --> or <-- -->)
    if s_strand == 1:
        promoter_start = max(int(coding_intervals[-2][0]), 
                             int(coding_intervals[-2][1]))
    else:
        promoter_start = max(int(coding_intervals[1][0]), 
                     int(coding_intervals[1][1]))
    #Everything upstream of the promoter start is clipped off the 
    # sequence and the substring is returned.
    return_seq = str(sequence[promoter_start : ])
    #Appends information to record description
    if len(return_seq) >= min_size:
        return SeqRecord(Seq.Seq(return_seq), id= return_id, description = return_id+"_"+str(s_start)+"_"+str(s_stop))


def id_below_maxid_perc(el1, el2, max_percent_id):
    
    """ Aligns two sequence elements and determines whether they are
        more than %ID identical (false) or not (true)
        Scoring: Match:+2, Mismatch:-1, GapO: -2, GapE: -0.2
    """
    
#    print "Seq1 length: " + str(len(el1.seq)) + ' ' + el1.id
#    print "Seq2 length: " + str(len(el2.seq)) + ' ' + el2.id
    al=pairwise2.align.globalms(el1.seq, el2.seq, 2, 0, -2, -.5,\
                                one_alignment_only=True, \
                                penalize_end_gaps=False)
    
    #print al
    matches=0
    gapless=0
    #for each position in the alignment
    for ch_pair in zip(al[0][0],al[0][1]):
        #if this is a non-gapped position
        if '-' not in ch_pair:
            #if it's a match, count it
            if ch_pair[0]==ch_pair[1]:
                matches=matches+1
            gapless=gapless+1
        
    perID = float(matches)/float(gapless)
    
#    print "Matches: ", matches
#    print "Gapless: ", gapless
#    print "%ID: ", perID
    
    #return true or false depending on percent identity
    if perID*100<=float(max_percent_id):
        return(True)
    else:
        return(False)


def remove_redundants(promoters_list, max_percent_identity):
    
    """ Takes a list of promoter sequences and returns a list of promoters with
        <= max_percent_identity.
        The identitiy values are computed using id_below_maxid_perc function defined
        above
    """
    
    removed_ids = []
    nr_promoters = []
    
    for i in range(0, len(promoters_list)-1):
        if promoters_list[i].description not in removed_ids:
            nr_promoters.append(promoters_list[i])
            for j in range(i+1, len(promoters_list)):
                if id_below_maxid_perc(promoters_list[i],promoters_list[j],max_percent_identity) == False:
                    removed_ids.append(promoters_list[j].description)
    
    return nr_promoters
                    

def memesearch(fastafile, nmotifs, minw, maxw, maxsites):
    
    """
        Takes a FASTA file and MEME parameters to perform the motif discovery by
        running MEME locally
    """
    
    bashCommand = "meme "+fastafile+" -dna -nostatus -time 18000 -mod anr -nmotifs "+str(nmotifs)+" -minw "+str(minw)+" -maxw "+str(maxw)+" -revcomp -maxsites "+str(maxsites)+" -evt 1e-30 -pal -oc "+fastafile.split(".")[0]
    subprocess.call(bashCommand.split())

In [4]:
# Load the configuration file

with open("test_input.json") as json_conf : 
    conf = json.load(json_conf)

In [ ]:
# Tell to who I am
#Entrez.email = conf["email"]
#Entrez.api_key = conf["api_key"]

In [4]:
#Open a file containing desired protein ids
filename = conf["cog_file"]
with open(filename) as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids] 

In [19]:
query_ids

['WP_002976456.1',
 'WP_002977359.1',
 'WP_002978128.1',
 'WP_002979859.1',
 'WP_002980971.1',
 'WP_002981428.1',
 'WP_002982566.1',
 'WP_004295675.1',
 'WP_004301910.1',
 'WP_004303213.1',
 'WP_004304717.1',
 'WP_004585519.1',
 'WP_005679415.1',
 'WP_005800434.1',
 'WP_005805768.1',
 'WP_005809884.1',
 'WP_005846380.1',
 'WP_005875237.1',
 'WP_006258617.1',
 'WP_006259902.1',
 'WP_007215664.1',
 'WP_007845880.1',
 'WP_008465565.1',
 'WP_008583029.1',
 'WP_008583608.1',
 'WP_008587938.1',
 'WP_008625890.1',
 'WP_008656253.1',
 'WP_008760075.1',
 'WP_008768057.1',
 'WP_009017772.1',
 'WP_009091290.1',
 'WP_009091498.1',
 'WP_009228497.1',
 'WP_009598553.1',
 'WP_010992752.1',
 'WP_011403084.1',
 'WP_011404287.1',
 'WP_011405594.1',
 'WP_011584878.1',
 'WP_011585064.1',
 'WP_011586424.1',
 'WP_011710922.1',
 'WP_011962560.1',
 'WP_012022970.1',
 'WP_012025549.1',
 'WP_012025745.1',
 'WP_012779936.1',
 'WP_012781528.1',
 'WP_012790878.1',
 'WP_012791024.1',
 'WP_012794435.1',
 'WP_0127947

In [20]:
#Create a dictionary using query_ids hits as keys and their location, species,
#and priority scores as values
ortholog_summary_dict = {}
for acc in query_ids:
    genome_record_retrieval_result = genome_record_retrieval(acc, 2)
    if genome_record_retrieval_result != None:
        ortholog_summary_dict[acc] = genome_record_retrieval_result

/home/nastya/miniconda3/envs/r4_env/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


IPG record retrieval error: WP_119755061.1


In [21]:
ortholog_summary_dict

{'WP_002976456.1': {'acc': 'NZ_LR134289.1',
  'start': '2855013',
  'stop': '2856110',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum'},
 'WP_002977359.1': {'acc': 'NZ_LR134289.1',
  'start': '2294568',
  'stop': '2295713',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum'},
 'WP_002978128.1': {'acc': 'NZ_LR134289.1',
  'start': '1616530',
  'stop': '1617672',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum'},
 'WP_002979859.1': {'acc': 'NZ_LR134289.1',
  'start': '436197',
  'stop': '437345',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum'},
 'WP_002980971.1': {'acc': 'NZ_LR134289.1',
  'start': '5232193',
  'stop': '5233335',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum'},
 'WP_002981428.1': {'acc': 'NZ_LR215967.1',
  'start': '4439542',
  'stop': '4440690',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_indologenes'},
 'WP_002982566.1': {'acc': 'NZ_LR134289.1',
  's

In [22]:
#Add the promoter information into the previous ortholog_summary_dict
for protein_id in ortholog_summary_dict.keys():
    current_promoter = promoter_retrieval(ortholog_summary_dict[protein_id]['acc'] ,ortholog_summary_dict[protein_id]['start'],ortholog_summary_dict[protein_id]['stop'],ortholog_summary_dict[protein_id]['strand'])
    if current_promoter:
        ortholog_summary_dict[protein_id]['prom'] = current_promoter

In [23]:
ortholog_summary_dict

{'WP_002976456.1': {'acc': 'NZ_LR134289.1',
  'start': '2855013',
  'stop': '2856110',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum',
  'prom': SeqRecord(seq=Seq('aatgcaggaagctggaagattgaagctggaagtagtttatgcactaagaagcct...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_2854763_2855015', dbxrefs=[])},
 'WP_002977359.1': {'acc': 'NZ_LR134289.1',
  'start': '2294568',
  'stop': '2295713',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum',
  'prom': SeqRecord(seq=Seq('tgctataaattatagcaacaaatatacaattttttgctacaaatcgtgataatt...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_2294318_2294570', dbxrefs=[])},
 'WP_002978128.1': {'acc': 'NZ_LR134289.1',
  'start': '1616530',
  'stop': '1617672',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Chryseobacterium_gleum',
  'prom': SeqRecord(seq=Seq('tgatacaatttgtatctacaaatatataaaaaatgatacaattagtatctaatt...atg'), id='p_NZ_LR134289.1', name='<unknown n

In [24]:
#download the promoters and remove redundant promoters using % identity
promoters = []
for key in ortholog_summary_dict.keys():
    try:
        promoters.append(ortholog_summary_dict[key.split("__")[0]]['prom'])
    except KeyError:
        continue
    
promoters_nr = remove_redundants(promoters,conf["nr_identity"])

In [25]:
promoters_nr

[SeqRecord(seq=Seq('aatgcaggaagctggaagattgaagctggaagtagtttatgcactaagaagcct...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_2854763_2855015', dbxrefs=[]),
 SeqRecord(seq=Seq('tgctataaattatagcaacaaatatacaattttttgctacaaatcgtgataatt...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_2294318_2294570', dbxrefs=[]),
 SeqRecord(seq=Seq('tttcttaatgcatcactaaataatgaaatattttatatatttaagtgttcgatt...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_435947_436199', dbxrefs=[]),
 SeqRecord(seq=Seq('cttgacgtgttttatttaaattttagttcaaacaataacaaagaagttaaatga...atg'), id='p_NZ_LR134289.1', name='<unknown name>', description='p_NZ_LR134289.1_3763354_3763606', dbxrefs=[]),
 SeqRecord(seq=Seq('cccccaaagatattctcaataaaatatatttcatttattcggggatttgtagta...atg'), id='p_NZ_GL945021.1', name='<unknown name>', description='p_NZ_GL945021.1_345608_345860', dbxrefs=[]),
 SeqRecord(seq=Seq('aatattgaagtatatagtactgtgatataaacttttcag

In [60]:
#create a directory to save MEME results
bashCommand = "mkdir MEME_Output"
subprocess.call(bashCommand.split())

#MEME discovery only in nodes with more than 10 non-redundant promoters
if len(promoters_nr) >= 10:
    #save the non-redundant promoters into a FASTA file
    out_handle_F = open("MEME_Output/nr_promoters.fasta", "w")
    for seqrecord in promoters_nr:
        out_handle_F.write(">"+"_".join(seqrecord.description.split(" ")[1:])+"_"+seqrecord.description.split(" ")[0]+"\n")
        out_handle_F.write(str(seqrecord.seq)+"\n")
    out_handle_F.close()
    #motif discovery with MEME
    #memesearch("MEME_Output/nr_promoters.fasta",10,conf["meme_minsize"],conf["meme_maxsize"],1000)

FileNotFoundError: [Errno 2] No such file or directory: 'meme'

export PATH=$HOME/meme/bin:$PATH
meme nr_promoters.fasta -dna -nostatus -time 18000 -mod anr -nmotifs 10 -minw 12 -maxw 26 -revcomp -maxsites 1000 -evt 1e-30 -pal -oc nr_promoters

In [4]:
import re

In [93]:
gene = 'dinB'
with open('try.txt', 'r') as file:
    for line in file:
        if f'[gene={gene}]' in line:
            a = re.findall(r"protein_id=\w*\.\w", line)
            print(str(a))

['protein_id=WP_018108812.1']


In [6]:
genes = ['dinB']#, 'uvrA','uvrB','recN','ruvA','polA','nusA','dnaK','recA']
listd = os.listdir('/home/nastya//Bacteroides _uniformis_annotations/genome_assemblies_cds_fasta/ncbi-genomes-2022-09-09/')
for gene in genes:
    for files in listd:
        with open(f'/home/nastya//Bacteroides _uniformis_annotations/genome_assemblies_cds_fasta/ncbi-genomes-2022-09-09/{files}', 'r') as file:
            for line in file:
                if f'[gene={gene}]' in line:
                    a = re.findall(r"protein_id=\w*\.\w", line)
                    with open(f'{gene}_1k_ids.txt', 'a') as out:
                        try:
                            out.write(f'''{str(a).strip("[]'").split('=')[1]}\n''')
                        except:
                            print(a)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [5]:
genes = ['recA']#, 'uvrA','uvrB','recN','ruvA','polA','nusA','dnaK','recA']
listd = os.listdir('/home/nastya//Bacteroides _uniformis_annotations/fasta_ncbi/')
for gene in genes:
    for files in listd:
        with open(f'/home/nastya//Bacteroides _uniformis_annotations/fasta_ncbi/{files}', 'r') as file:
            for line in file:
                if f'[gene={gene}]' in line:
                    a = re.findall(r"protein_id=\w*\.\w", line)
                    with open(f'{gene}_only_ids.txt', 'a') as out:
                        try:
                            out.write(f'''{str(a).strip("[]'").split('=')[1]}\n''')
                        except:
                            print(a)

[]
[]
[]


['lexA','uvrA','uvrB','recN','ruvA','polA','nusA','dnaK','recA','dinB']

In [5]:
#Open a file containing desired protein ids
gene = 'recA'
filename = f'{gene}_only_ids.txt'
with open(filename) as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [26]:
#Open a file containing desired protein ids
gene = 'umuD'
filename = f'umuD_Bacteroidetes_ids.txt'
with open(filename) as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [6]:
len(query_ids)

1742

In [7]:
#Create a dictionary using query_ids hits as keys and their location, species,
#and priority scores as values
ortholog_summary_dict = {}
for acc in query_ids:
    genome_record_retrieval_result = genome_record_retrieval(acc, 2)
    if genome_record_retrieval_result != None:
        ortholog_summary_dict[acc] = genome_record_retrieval_result

/home/nastya/miniconda3/envs/r4_env/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [8]:
ortholog_summary_dict

{'WP_005924339.1': {'acc': 'NZ_KB905466.1',
  'start': '623530',
  'stop': '624567',
  'strand': '+',
  'p_score': 6.5,
  'org': 'Bacteroides_salyersiae_WAL_10018_=_DSM_18765_=_JCM_12988'},
 'WP_007665644.1': {'acc': 'NZ_CP064941.1',
  'start': '3382327',
  'stop': '3383364',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Bacteroides_intestinalis'},
 'WP_005785787.1': {'acc': 'CP083688.1',
  'start': '3918029',
  'stop': '3919072',
  'strand': '-',
  'p_score': 6,
  'org': 'Bacteroides_fragilis'},
 'WP_229036781.1': {'acc': 'NZ_OU901070.1',
  'start': '2049831',
  'stop': '2050823',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Bacteroides_ovatus'},
 'WP_005652006.1': {'acc': 'FNOD01000010.1',
  'start': '89448',
  'stop': '90485',
  'strand': '+',
  'p_score': 0,
  'org': 'Bacteroides_stercoris'},
 'WP_004292863.1': {'acc': 'NZ_GL622538.1',
  'start': '459125',
  'stop': '460120',
  'strand': '-',
  'p_score': 6.5,
  'org': 'Bacteroides_eggerthii_1_2_48FAA'},
 'WP_227985691.1': {'acc': '

In [ ]:
#Add the promoter information into the previous ortholog_summary_dict
for protein_id in ortholog_summary_dict.keys():
    try:
        current_promoter = promoter_retrieval(ortholog_summary_dict[protein_id]['acc'] ,ortholog_summary_dict[protein_id]['start'],ortholog_summary_dict[protein_id]['stop'],ortholog_summary_dict[protein_id]['strand'])
        if current_promoter:
            ortholog_summary_dict[protein_id]['prom'] = current_promoter
    except:
        print(protein_id)

Promoter retrieval error: NZ_KB905466.1
WP_005924339.1
Promoter retrieval error: NZ_CP064941.1
WP_007665644.1


In [11]:
#download the promoters and remove redundant promoters using % identity
promoters = []
for key in ortholog_summary_dict.keys():
    try:
        promoters.append(ortholog_summary_dict[key.split("__")[0]]['prom'])
    except KeyError:
        continue
    
promoters_nr = remove_redundants(promoters,conf["nr_identity"])

In [14]:
#create a directory to save MEME results
bashCommand = f"mkdir MEME_Output_{gene}"
subprocess.call(bashCommand.split())

#MEME discovery only in nodes with more than 10 non-redundant promoters
if len(promoters_nr) >= 10:
    #save the non-redundant promoters into a FASTA file
    out_handle_F = open(f"MEME_Output_{gene}/nr_promoters.fasta", "w")
    for seqrecord in promoters_nr:
        out_handle_F.write(">"+"_".join(seqrecord.description.split(" ")[1:])+"_"+seqrecord.description.split(" ")[0]+"\n")
        out_handle_F.write(str(seqrecord.seq)+"\n")
    out_handle_F.close()
    #motif discovery with MEME
    #memesearch("MEME_Output/nr_promoters.fasta",10,conf["meme_minsize"],conf["meme_maxsize"],1000)

In [133]:
print('A')

A


In [ ]:
gene = 'dinB'
filename = f'{gene}_only_ids.txt'
with open(filename) as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]
ortholog_summary_dict = {}
for acc in query_ids:
    genome_record_retrieval_result = genome_record_retrieval(acc, 2)
    if genome_record_retrieval_result != None:
        ortholog_summary_dict[acc] = genome_record_retrieval_result
for protein_id in ortholog_summary_dict.keys():
    current_promoter = promoter_retrieval(ortholog_summary_dict[protein_id]['acc'] ,ortholog_summary_dict[protein_id]['start'],ortholog_summary_dict[protein_id]['stop'],ortholog_summary_dict[protein_id]['strand'])
    if current_promoter:
        ortholog_summary_dict[protein_id]['prom'] = current_promoter
promoters = []
for key in ortholog_summary_dict.keys():
    try:
        promoters.append(ortholog_summary_dict[key.split("__")[0]]['prom'])
    except KeyError:
        continue
promoters_nr = remove_redundants(promoters,conf["nr_identity"])
#create a directory to save MEME results
bashCommand = f"mkdir MEME_Output_{gene}_aa"
subprocess.call(bashCommand.split())

#MEME discovery only in nodes with more than 10 non-redundant promoters
if len(promoters_nr) >= 10:
    #save the non-redundant promoters into a FASTA file
    out_handle_F = open(f"MEME_Output_{gene}_aa/nr_promoters.fasta", "w")
    for seqrecord in promoters_nr:
        out_handle_F.write(">"+"_".join(seqrecord.description.split(" ")[1:])+"_"+seqrecord.description.split(" ")[0]+"\n")
        out_handle_F.write(str(seqrecord.seq)+"\n")
    out_handle_F.close()

In [147]:
promoters_nr = promoters
#create a directory to save MEME results
bashCommand = f"mkdir MEME_Output_{gene}_2"
subprocess.call(bashCommand.split())

#MEME discovery only in nodes with more than 10 non-redundant promoters
if len(promoters_nr) >= 10:
    #save the non-redundant promoters into a FASTA file
    out_handle_F = open(f"MEME_Output_{gene}_2/nr_promoters.fasta", "w")
    for seqrecord in promoters_nr:
        out_handle_F.write(">"+"_".join(seqrecord.description.split(" ")[1:])+"_"+seqrecord.description.split(" ")[0]+"\n")
        out_handle_F.write(str(seqrecord.seq)+"\n")
    out_handle_F.close()

In [140]:
with open('umuD_all_ids.txt') as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [141]:
len(query_ids)

413

In [138]:
with open('umuD_Bacteroidetes_ids.txt') as f:
    a = f.readlines()
a = [x.strip() for x in a]

In [139]:
len(a)

302

In [143]:
inter = set(a)-set(query_ids)
inter

{'WP_009598742.1',
 'WP_010956050.1',
 'WP_010956286.1',
 'WP_010956476.1',
 'WP_011403931.1',
 'WP_011584877.1',
 'WP_011585264.1',
 'WP_011586425.1',
 'WP_011962561.1',
 'WP_012024039.1',
 'WP_012055704.1',
 'WP_012458045.1',
 'WP_012458292.1',
 'WP_012458488.1',
 'WP_012458492.1',
 'WP_012845191.1',
 'WP_013065178.1',
 'WP_013065665.1',
 'WP_013446446.1',
 'WP_013619408.1',
 'WP_013622463.1',
 'WP_013763228.1',
 'WP_013769050.1',
 'WP_013816132.1',
 'WP_013816271.1',
 'WP_013816657.1',
 'WP_013869467.1',
 'WP_013929570.1',
 'WP_013930886.1',
 'WP_014033044.1',
 'WP_014072512.1',
 'WP_014085094.1',
 'WP_014201447.1',
 'WP_014221547.1',
 'WP_014222859.1',
 'WP_014387208.1',
 'WP_014775772.1',
 'WP_014776103.1',
 'WP_015025842.1',
 'WP_015031119.1',
 'WP_015481075.1',
 'WP_022333149.1',
 'WP_025603823.1',
 'WP_025606040.1',
 'WP_025606869.1',
 'WP_028291585.1',
 'WP_035640312.1',
 'WP_036908457.1',
 'WP_038460437.1',
 'WP_038654215.1',
 'WP_040602640.1',
 'WP_041618076.1',
 'WP_0435519

In [144]:
inter = set(query_ids)-set(a)
inter

{'WP_005794801.1',
 'WP_005938311.1',
 'WP_006744281.1',
 'WP_006745627.1',
 'WP_007215807.1',
 'WP_008768414.1',
 'WP_011202621.1',
 'WP_018108948.1',
 'WP_024997049.1',
 'WP_042369637.1',
 'WP_044095494.1',
 'WP_044267288.1',
 'WP_044269883.1',
 'WP_051089642.1',
 'WP_054960679.1',
 'WP_055217582.1',
 'WP_055228780.1',
 'WP_060408184.1',
 'WP_068855440.1',
 'WP_073398722.1',
 'WP_075317389.1',
 'WP_087207130.1',
 'WP_087209068.1',
 'WP_087247715.1',
 'WP_087251027.1',
 'WP_087398148.1',
 'WP_118038407.1',
 'WP_118403534.1',
 'WP_118419698.1',
 'WP_118496696.1',
 'WP_118525160.1',
 'WP_122330021.1',
 'WP_140400143.1',
 'WP_149950387.1',
 'WP_154042805.1',
 'WP_161799601.1',
 'WP_163177562.1',
 'WP_165861957.1',
 'WP_175393085.1',
 'WP_175631586.1',
 'WP_181996046.1',
 'WP_181999657.1',
 'WP_182423322.1',
 'WP_182423832.1',
 'WP_195281948.1',
 'WP_195310625.1',
 'WP_195342348.1',
 'WP_195359460.1',
 'WP_195407137.1',
 'WP_195558522.1',
 'WP_195595186.1',
 'WP_196244780.1',
 'WP_1969835

In [4]:
with open('dinB_all_ids.txt') as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [23]:
with open('dinB_Bacteroidetes_ids.txt') as f:
    a = f.readlines()
a = [x.strip() for x in a]

In [5]:
query_ids

['WP_025819683.1',
 'WP_008760075.1',
 'WP_008760075.1',
 'WP_118476983.1',
 'WP_008764564.1',
 'WP_008768057.1',
 'WP_032843930.1',
 'WP_117740203.1',
 'WP_004289681.1',
 'WP_004325039.1',
 'WP_032556403.1',
 'WP_061448094.1',
 'WP_005831892.1',
 'WP_117588465.1',
 'WP_032587853.1',
 'WP_074557371.1',
 'WP_005831892.1',
 'WP_032529860.1',
 'WP_035448989.1',
 'WP_117985771.1',
 'WP_032529860.1',
 'WP_117955488.1',
 'WP_008768057.1',
 'WP_022042486.1',
 'WP_004301910.1',
 'WP_016662554.1',
 'WP_106484504.1',
 'WP_032840532.1',
 'WP_004301910.1',
 'WP_004301910.1',
 'WP_004301910.1',
 'WP_032813576.1',
 'WP_044469011.1',
 'WP_117514951.1',
 'WP_005831892.1',
 'WP_027325556.1',
 'WP_008768057.1',
 'WP_004301910.1',
 'WP_004289681.1',
 'WP_122143609.1',
 'WP_080979882.1',
 'WP_008760075.1',
 'WP_004301910.1',
 'WP_055279139.1',
 'WP_004301910.1',
 'WP_005831892.1',
 'WP_022042486.1',
 'WP_007757999.1',
 'WP_004301910.1',
 'WP_010536998.1',
 'WP_004301910.1',
 'WP_007765421.1',
 'WP_0043019

In [25]:
a

['WP_002976456.1',
 'WP_002977359.1',
 'WP_002978128.1',
 'WP_002979859.1',
 'WP_002980971.1',
 'WP_002981428.1',
 'WP_002982566.1',
 'WP_004295675.1',
 'WP_004301910.1',
 'WP_004303213.1',
 'WP_004304717.1',
 'WP_004585519.1',
 'WP_005679415.1',
 'WP_005800434.1',
 'WP_005805768.1',
 'WP_005809884.1',
 'WP_005846380.1',
 'WP_005875237.1',
 'WP_006258617.1',
 'WP_006259902.1',
 'WP_007215664.1',
 'WP_007845880.1',
 'WP_008465565.1',
 'WP_008583029.1',
 'WP_008583608.1',
 'WP_008587938.1',
 'WP_008625890.1',
 'WP_008656253.1',
 'WP_008760075.1',
 'WP_008768057.1',
 'WP_009017772.1',
 'WP_009091290.1',
 'WP_009091498.1',
 'WP_009228497.1',
 'WP_009598553.1',
 'WP_010992752.1',
 'WP_011403084.1',
 'WP_011404287.1',
 'WP_011405594.1',
 'WP_011584878.1',
 'WP_011585064.1',
 'WP_011586424.1',
 'WP_011710922.1',
 'WP_011962560.1',
 'WP_012022970.1',
 'WP_012025549.1',
 'WP_012025745.1',
 'WP_012779936.1',
 'WP_012781528.1',
 'WP_012790878.1',
 'WP_012791024.1',
 'WP_012794435.1',
 'WP_0127947

In [3]:
import os
import re
import logging
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
with open('s7_lexa.csv') as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [19]:
for gene_id in query_ids:
    try:
        url = "https://www.ncbi.nlm.nih.gov/protein/{}"
        r = requests.get(url.format(gene_id))  # Access url page of input platform
        data = r.text  # Get whole page in HTML format
        soup = str(BeautifulSoup(data, "html.parser"))  # Pull data from HTML format
        req = re.search(
            r"<title>.*?/[.*/]", soup, re.DOTALL | re.MULTILINE
        )  # Get data from Title field
        result = ""  # Data extracted from req and/or req1 fields will be put into result
        if req:
            result = str(req.group())
        res = re.search(r"(?<=\[).+?(?=\])", result).group()
        with open('species_lexa.txt', 'a') as out:
            out.write(f'{res}\n')
    except:
        print(gene_id)

WP_013750282.1
WP_139065562.1
WP_146791046.1
WP_147033392.1
WP_149069274.1


In [20]:
ab=[]
with open('species_lexa.txt') as f:
    names = f.readlines()
my_list = [x.strip() for x in names]
my_un = list(set(my_list))
for i in my_un:
    if i !='Bacteroides':
        ab.append(i)

In [21]:
ab

['Pontibacter pudoricolor',
 'Hymenobacter sp. APR13',
 'Flavisolibacter tropicus',
 'Pontibacter akesuensis',
 'Aequorivita sp. H23M31',
 'Hymenobacter sp. HDW8',
 'Maribacter',
 'Dyadobacter fermentans',
 'Sphingobacterium sp. ML3W',
 'Flagellimonas maritima',
 'Cytophaga hutchinsonii',
 'Maribacter sp. HTCC2170',
 'Chitinophaga sp. XS-30',
 'Muricauda ruestringensis',
 'Euzebyella marina',
 'Pedobacter',
 'Chitinophaga pinensis',
 'Hymenobacter sedentarius',
 'Maribacter hydrothermalis',
 'Belliella baltica',
 'Hymenobacter jejuensis',
 'Chitinophaga deserti',
 'Aquimarina sp. BL5',
 'Chitinophaga oryzae',
 'Dokdonia donghaensis',
 'Hymenobacter sp. DG25A',
 'Pedobacter schmidteae',
 'Olivibacter',
 'Niabella ginsenosidivorans',
 'Maribacter cobaltidurans',
 'Pedobacter cryoconitis',
 'Arenibacter algicola',
 'Maribacter sp. MJ134',
 'Hymenobacter sp. DG01',
 'Salegentibacter',
 'Marivirga tractuosa',
 'Filimonas lacunae',
 'Chitinophaga caeni',
 'Dokdonia sp. Dokd-P16',
 'Arachidic

In [35]:
with open('res_list.txt') as f:
    a = f.readlines()
a = [x.strip() for x in a]

In [36]:
for gene_id in a:
    try:
        url = "https://www.ncbi.nlm.nih.gov/protein/{}"
        r = requests.get(url.format(gene_id))  # Access url page of input platform
        data = r.text  # Get whole page in HTML format
        soup = str(BeautifulSoup(data, "html.parser"))  # Pull data from HTML format
        req = re.search(
            r"<title>.*?/[.*/]", soup, re.DOTALL | re.MULTILINE
        )  # Get data from Title field
        result = ""  # Data extracted from req and/or req1 fields will be put into result
        if req:
            result = str(req.group())
        res = re.search(r"(?<=\[).+?(?=\])", result).group()
        with open('species_article.txt', 'a') as out:
            out.write(f'{res}\n')
    except:
        print(gene_id)

WP_013768398.1
WP_086546885.1
WP_086546896.1
WP_086548355.1
WP_153509908.1


In [31]:
ab=[]
with open('species_mine.txt') as f:
    names = f.readlines()
my_list = [x.strip() for x in names]
my_un = list(set(my_list))
for i in my_un:
    if i !='Bacteroides':
        ab.append(i)

In [33]:
len(ab)

67

In [42]:
ab

['Bacteroides sp. NM69_E16B',
 'Bacteroides gallinarum',
 'Bacteroides sp. NSJ-90',
 'Bacteroides sp. 214',
 'unclassified Bacteroides',
 'Bacteroides sp. 51',
 'Bacteroides eggerthii',
 'Bacteroides sp. D2',
 'Phocaeicola dorei',
 'Bacteroides sp. L10-4',
 'Bacteroides rodentium',
 'Bacteroides thetaiotaomicron',
 'Bacteroidaceae',
 'Bacteroides zoogleoformans',
 'Bacteroides sp. 11BF',
 'Bacteroides ndongoniae',
 'Bacteroides sp. An322',
 'Bacteroides luti',
 'Bacteroides stercoris',
 'Bacteroides finegoldii',
 'Bacteroides sp. AF37-16AC',
 'Bacteroides sp. GM023',
 'Bacteroides uniformis',
 'Bacteroides caccae',
 'Bacteroides acidifaciens',
 'Bacteroides faecichinchillae',
 'Bacteroides congonensis',
 'Bacteroides caecicola',
 'Bacteroides sp. 224',
 'Bacteroides pyogenes',
 'Bacteroides salyersiae',
 'Bacteroides faecis',
 'Bacteroides sp. ET71',
 'Bacteroides nordii',
 'Bacteroides oleiciplenus',
 'Bacteroides cellulosilyticus',
 'Bacteroides neonati',
 'Bacteroides ihuae',
 'Bact

In [20]:
bc=[]
with open('species_article.txt') as f:
    names = f.readlines()
my_list = [x.strip() for x in names]
my_un = list(set(my_list))
for i in my_un:
    if i !='Bacteroides':
        bc.append(i)

In [38]:
len(bc)

392

In [40]:
ab

['Bacteroides sp. NM69_E16B',
 'Bacteroides gallinarum',
 'Bacteroides sp. NSJ-90',
 'Bacteroides sp. 214',
 'unclassified Bacteroides',
 'Bacteroides sp. 51',
 'Bacteroides eggerthii',
 'Bacteroides sp. D2',
 'Phocaeicola dorei',
 'Bacteroides sp. L10-4',
 'Bacteroides rodentium',
 'Bacteroides thetaiotaomicron',
 'Bacteroidaceae',
 'Bacteroides zoogleoformans',
 'Bacteroides sp. 11BF',
 'Bacteroides ndongoniae',
 'Bacteroides sp. An322',
 'Bacteroides luti',
 'Bacteroides stercoris',
 'Bacteroides finegoldii',
 'Bacteroides sp. AF37-16AC',
 'Bacteroides sp. GM023',
 'Bacteroides uniformis',
 'Bacteroides caccae',
 'Bacteroides acidifaciens',
 'Bacteroides faecichinchillae',
 'Bacteroides congonensis',
 'Bacteroides caecicola',
 'Bacteroides sp. 224',
 'Bacteroides pyogenes',
 'Bacteroides salyersiae',
 'Bacteroides faecis',
 'Bacteroides sp. ET71',
 'Bacteroides nordii',
 'Bacteroides oleiciplenus',
 'Bacteroides cellulosilyticus',
 'Bacteroides neonati',
 'Bacteroides ihuae',
 'Bact

In [22]:
for i in bc:
    print(i)

Aquimarina sp. AD10
Flammeovirgaceae bacterium 311
Rufibacter radiotolerans
Tamlana carrageenivorans
Petrimonas mucosa
Muriicola soli
Pseudobacter ginsenosidimutans
Sphingobacterium thalpophilum
Aquimarina sp. BL5
Kordia sp. SMS9
Flavobacteriaceae bacterium F202Z8
Flavobacterium psychrophilum
Capnocytophaga endodontalis
Flammeovirga pectinis
Alkalitalea saponilacus
Croceibacter atlanticus
Bacteroidaceae
Chryseobacterium sp. StRB126
Nonlabens sp. MIC269
Chryseobacterium glaciei
Mucilaginibacter xinganensis
Phocaeicola dorei
unclassified Capnocytophaga
Sediminicola sp. YIK13
Mucilaginibacter
Zobellia
Chryseobacterium sp. G0162
Flavobacterium
Oceanihabitans sp. IOP_32
Flavobacteriaceae bacterium UJ101
Winogradskyella sp. J14-2
Spirosoma rigui
Bacteroides uniformis
Pedobacter heparinus
Dyadobacter fermentans
Rufibacter tibetensis
Weeksella virosa
Flavobacterium sp. Sr18
Dokdonia sp. MED134
Elizabethkingia anophelis
Capnocytophaga cynodegmi
Antarcticibacterium flavum
Flavobacterium sp. KBS0

In [41]:
count = 0
for i in bc:
    if 'Bacteroides' in i:
        count+=1
count

11

In [44]:
set(bc)&set(ab)

{'Bacteroidaceae',
 'Bacteroides caccae',
 'Bacteroides caecimuris',
 'Bacteroides cellulosilyticus',
 'Bacteroides fragilis',
 'Bacteroides helcogenes',
 'Bacteroides intestinalis',
 'Bacteroides ovatus',
 'Bacteroides sp. CBA7301',
 'Bacteroides thetaiotaomicron',
 'Bacteroides uniformis',
 'Bacteroides zoogleoformans',
 'Phocaeicola dorei'}

In [4]:
with open('dinB_1k_ids.txt') as f:
    query_ids = f.readlines()
query_ids = [x.strip() for x in query_ids]

In [5]:
query_ids

['WP_087939150.1',
 'WP_087939212.1',
 'WP_155211402.1',
 'WP_155212550.1',
 'WP_235954346.1',
 'WP_171606598.1',
 'WP_235832017.1',
 'WP_051227743.1',
 'WP_146930371.1',
 'WP_119437993.1',
 'WP_243646060.1',
 'WP_073121375.1',
 'WP_072999877.1',
 'WP_019942314.1',
 'WP_019943963.1',
 'WP_035133251.1',
 'WP_136840772.1',
 'WP_008626466.1',
 'WP_187656435.1',
 'WP_114752244.1',
 'WP_232835028.1',
 'WP_002993460.1',
 'WP_052646396.1',
 'WP_041557247.1',
 'WP_109673982.1',
 'WP_109674142.1',
 'WP_008201631.1',
 'WP_008202096.1',
 'WP_027201330.1',
 'WP_143409968.1',
 'WP_243735775.1',
 'WP_205644689.1',
 'WP_205644946.1',
 'WP_121459930.1',
 'WP_151107195.1',
 'WP_103004099.1',
 'WP_031525861.1',
 'WP_229206766.1',
 'WP_111539703.1',
 'WP_026710108.1',
 'WP_241459835.1',
 'WP_026899292.1',
 'WP_236139614.1',
 'WP_236136170.1',
 'WP_089853352.1',
 'WP_089056014.1',
 'WP_089053424.1',
 'WP_173587492.1',
 'WP_169227657.1',
 'WP_169228528.1',
 'WP_141420301.1',
 'WP_141421854.1',
 'WP_0356549

In [7]:
query_ids = ['WP_235954346.1',
'WP_171606598.1',
'WP_235832017.1',
'WP_148908705.1',
'WP_159635276.1']

In [8]:
for gene_id in query_ids:
    try:
        url = "https://www.ncbi.nlm.nih.gov/protein/{}"
        r = requests.get(url.format(gene_id))  # Access url page of input platform
        data = r.text  # Get whole page in HTML format
        soup = str(BeautifulSoup(data, "html.parser"))  # Pull data from HTML format
        req = re.search(
            r"<title>.*?/[.*/]", soup, re.DOTALL | re.MULTILINE
        )  # Get data from Title field
        result = ""  # Data extracted from req and/or req1 fields will be put into result
        if req:
            result = str(req.group())
        res = re.search(r"(?<=\[).+?(?=\])", result).group()
        with open('species_1k.txt', 'a') as out:
            out.write(f'{res}\n')
    except:
        print(gene_id)

WP_148908705.1
WP_159635276.1


In [9]:
bc=[]
with open('species_1k.txt') as f:
    names = f.readlines()
my_list = [x.strip() for x in names]
my_un = list(set(my_list))
for i in my_un:
    if i !='Bacteroides':
        bc.append(i)

In [10]:
len(bc)

1577

In [11]:
bc

['Haliscomenobacter hydrossis',
 'Olleya',
 'Epilithonimonas zeae',
 'Draconibacterium halophilum',
 'Pedobacter foliorum',
 'Hymenobacter aerophilus',
 'Lewinella antarctica',
 'Portibacter lacus',
 'Cecembia calidifontis',
 'Dyadobacter jejuensis',
 'Gillisia hiemivivida',
 'Aquimarina megaterium',
 'Flavobacterium denitrificans',
 'Gelidibacter sediminis',
 'Zobellia nedashkovskayae',
 'Mucilaginibacter polytrichastri',
 'Luteibaculum oceani',
 'Niastella yeongjuensis',
 'Flavobacterium cutihirudinis',
 'Psychroflexus salarius',
 'Hymenobacter glacieicola',
 'Algibacter onchidii',
 'Tenacibaculum skagerrakense',
 'Nonlabens marinus',
 'Pontibacter russatus',
 'Hymenobacter fodinae',
 'Flavobacterium fluvii',
 'Alistipes communis',
 'Algoriphagus aquimaris',
 'Algoriphagus sanaruensis',
 'Elizabethkingia argenteiflava',
 'Prevotella amnii',
 'Salegentibacter maritimus',
 'Gelidibacter maritimus',
 'Chryseolinea serpens',
 'Flavobacterium jejuense',
 'Chryseobacterium populi',
 'Prevo

In [122]:
df = pd.DataFrame()
for file in os.listdir('metadata'):
    data = pd.read_csv(f'metadata/{file}', sep='\t')[['id','sample_description']]
    if len(df) ==0:
        df = data
    df = pd.concat([df, data], axis=0)

In [123]:
df

,id,sample_description
0,SRX7648631_SRR10987170,WT rep2 stat
0,SRX7648631_SRR10987170,WT rep2 stat
0,SRX7648635_SRR10987174,WT TEX rep3 ELP
0,SRX7648647_SRR10987186,BTnc035 complement rep2
0,SRX7648632_SRR10987171,WT rep3 stat
0,SRX7648634_SRR10987173,WT TEX rep2 ELP
0,SRX7648643_SRR10987182,WT rep2
0,SRX7648642_SRR10987181,WT rep1
0,SRX7648625_SRR10987164,WT rep2 ELP
0,SRX7648624_SRR10987163,WT rep1 ELP


SRX7648627_SRR10987166	WT rep1 MLP
SRX7648628_SRR10987167	WT rep2 MLP
SRX7648629_SRR10987168	WT rep3 MLP

SRX7648630_SRR10987169	WT rep1 stat
SRX7648631_SRR10987170	WT rep2 stat
SRX7648632_SRR10987171	WT rep3 stat

SRX7648636_SRR10987175	WT TEX rep1 MLP
SRX7648637_SRR10987176	WT TEX rep2 MLP
SRX7648638_SRR10987177	WT TEX rep3 MLP

SRX7648639_SRR10987178	WT TEX rep1 stat
SRX7648640_SRR10987179	WT TEX rep2 stat
SRX7648641_SRR10987180	WT TEX rep3 stat